### Sample CORE results

This is not the main notebook in this challenge. See `instructions.ipynb`

Show a handful of correct and incorrect results on the various CORE tasks. Currently only showing squad.

In [1]:
import os
import torch
import sys
import yaml
import json
import textwrap
import random
sys.path.append('../my_nanochat')
from my_nanochat.my_common import get_base_dir, autodetect_device_type, compute_init
from my_nanochat.my_checkpoint_manager import load_model
from my_nanochat.my_core_eval import evaluate_example, render_prompts_mc, render_prompts_lm, render_prompts_schema

In [2]:
device_type = autodetect_device_type() 
_, _, _, _, device = compute_init(device_type)
model, tokenizer, meta_data = load_model('base', model_tag='d32', device=device, phase='eval')
autocast_ctx = torch.amp.autocast(device_type=device_type, dtype=torch.bfloat16) if device_type == "cuda" else nullcontext()

Autodetected device type: cuda
loading the model from /home/ubuntu/mynanochat2/base_checkpoints/d32 with step 71680


/home/ubuntu/learn-nanochat/.venv/lib/python3.10/site-packages/torch/__init__.py:1617: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  _C._set_float32_matmul_precision(precision)


Building model with config: {'sequence_len': 2048, 'vocab_size': 65536, 'n_layer': 32, 'n_head': 16, 'n_kv_head': 16, 'n_embd': 2048}


In [3]:
def print_wrap(s, remove_newlines=False):
    if remove_newlines:
        s = s.replace("\n", " ")
    print(textwrap.fill(s, 90))

In [4]:
base_dir = get_base_dir()
eval_bundle_dir = os.path.join(base_dir, "eval_bundle")
config_path = os.path.join(eval_bundle_dir, "core.yaml")
data_base_path = os.path.join(eval_bundle_dir, "eval_data")
with open(config_path, 'r', encoding='utf-8') as f:
    config = yaml.safe_load(f)
tasks = config['icl_tasks']

In [30]:
def print_per_token_losses(losses, tokens):
    for i, (losses, tokens) in enumerate(zip(per_token_losses, target_tokens)):
        print(f"prompt {i}")
        print('losses:\t', '\t'.join([f"{loss:.4f}" for loss in losses]))
        print('token ids:\t ', '\t'.join([str(token_id) for token_id in tokens]))
        print('tokens:\t', '\t'.join([tokenizer.decode([token_id]) for token_id in tokens]))
        print()

In [32]:
tasks_to_inspect = ['squad', 'arc_easy', 'arc_challenge', 'bigbench_dyck_languages']
items_per_task = 20

for task in tasks:
    if task['label'] not in tasks_to_inspect:
        continue

    task_meta = {
        'task_type': task['icl_task_type'],
        'dataset_uri': task['dataset_uri'],
        'num_fewshot': task['num_fewshot'][0],
        'continuation_delimiter': task.get('continuation_delimiter', ' ')
    }
    
    task_type = task['icl_task_type']
    continuation_delimiter = task.get('continuation_delimiter', ' ')
    data_path = os.path.join(data_base_path, task['dataset_uri'])
    
    with open(data_path, 'r', encoding='utf-8') as f:
        data = [json.loads(line.strip()) for line in f]

        random.seed(5)
        for i in random.sample(range(0, len(data)), items_per_task):
            with autocast_ctx:
                result, prompts, mean_losses, per_token_losses, target_tokens = evaluate_example(i, model, tokenizer, data, device, task_meta, return_prompts_and_losses=True)
                print(f"{task['label']} item {i} is {'correct' if result else 'wrong'}\n")
                item = data[i]
                if task_type == 'language_modeling':
                    print("Here is the expected continuation and full prompt including the n-shot examples if applicable:\n")
                    print_wrap(f"Expected continuation: {item['continuation']}")
                    print()
                    prompt = prompts[0] # for LM we only care about this one
                    
                    print_wrap(prompt)

                    print("\nHere's what the model outputs with the full prompt, max_tokens=20, temperature=0:")
                    in_tokens = tokenizer.encode(prompt, prepend=tokenizer.get_bos_token_id())
                    out_tokens = list(model.generate(in_tokens, max_tokens=20, temperature=0))
                    print(tokenizer.decode(out_tokens))

                    for seed in range(5):
                        print(f"\nHere's what the model outputs with the full prompt, max_tokens=20, temperature=1, seed={seed}:")
                        in_tokens = tokenizer.encode(prompt, prepend=tokenizer.get_bos_token_id())
                        out_tokens = list(model.generate(in_tokens, max_tokens=20, temperature=1, seed=seed))
                        print(tokenizer.decode(out_tokens))
                    
                elif task_type == 'multiple_choice': 
                    print(f"The correct prompt is: {item['gold']}\n")

                    print("Here are the prompts but _without_ the n-shot examples even when applicable:\n")
                    prompts_without_nshot = render_prompts_mc(item, continuation_delimiter, [])
                    for j, prompt in enumerate(prompts_without_nshot):
                        print_wrap(f"prompt {j}: {prompt}")
                        print()

                    pred_idx = mean_losses.index(min(mean_losses))
                    print(f"The prediction from the model was prompt: {pred_idx}")
                    print("The mean losses (in prompt order) were", ', '.join([f"{ml:.4f}" for ml in mean_losses]))

                    print("\nThe per-token losses were (tab delimited to copy to google sheets):")
                    print_per_token_losses(per_token_losses, target_tokens)
                
                else:
                    assert False # TOOD other types
                
                print("\n=================================\n")

arc_easy item 1046 is wrong

The correct prompt is: 3

Here are the prompts but _without_ the n-shot examples even when applicable:

prompt 0: Question: Which explains how the epithelium offers protection to land-dwelling
vertebrates? Answer: Epithelium provides a rigid shell to prevent punctures.

prompt 1: Question: Which explains how the epithelium offers protection to land-dwelling
vertebrates? Answer: Epithelium has capillaries to resist iron deficiency.

prompt 2: Question: Which explains how the epithelium offers protection to land-dwelling
vertebrates? Answer: Epithelium insulates the vertebrate from hypothermia.

prompt 3: Question: Which explains how the epithelium offers protection to land-dwelling
vertebrates? Answer: Epithelium prevents dehydration in vertebrates.

The prediction from the model was prompt: 0
The mean losses (in prompt order) were 3.5148, 6.4196, 4.0085, 3.7062

The per-token losses were (tab delimited to copy to google sheets):
prompt 0
losses:	 3.3906	0.4